In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models
import os
import numpy as np


In [3]:
pwd

'/home/wsuser/work'

In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='oMGmiKgTzljtiNY3Mun5dF2KggJ_qxkEHJRiiCYrOec8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-tljphflac3cpyb'
object_key = 'fruit-dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [5]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [10]:
pwd

'/home/wsuser/work'

In [2]:
train_datagen = ImageDataGenerator (rescale = 1./255, shear_range= 0.2,zoom_range= 0.2, horizontal_flip = True)
test_datagen =ImageDataGenerator (rescale = 1)

In [15]:
x_train = train_datagen.flow_from_directory(
        r'D:\IBM\Dataset Plant Disease\fruit-dataset\fruit-dataset\train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse",
#         save_to_dir="C:\\Code\\potato-disease-classification\\training\\AugmentedImages"
)

Found 5384 images belonging to 6 classes.


In [11]:
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/fruit-dataset/train',target_size = (128,128), batch_size = 32, class_mode = 'categorical')
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/fruit-dataset/test',target_size = (128,128), batch_size = 32, class_mode = 'categorical')

Found 5384 images belonging to 6 classes.
Found 1686 images belonging to 6 classes.


In [10]:
pip install 'h5py==2.10.0' --force-reinstall

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'h5py==2.10.0'"


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

In [15]:
model=Sequential()

In [16]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [17]:
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 127008)            0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

In [19]:
model.add(Dense(6,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
len(x_train)

169

In [21]:
model.fit(x_train,steps_per_epoch=len(x_train),epochs=10)

Epoch 1/10
169/169 [==============================] - 56s 330ms/step - loss: 1.5752 - accuracy: 0.7400
Epoch 2/10
169/169 [==============================] - 55s 325ms/step - loss: 0.3001 - accuracy: 0.8990
Epoch 3/10
169/169 [==============================] - 52s 308ms/step - loss: 0.2587 - accuracy: 0.9097
Epoch 4/10
169/169 [==============================] - 55s 322ms/step - loss: 0.1890 - accuracy: 0.9363
Epoch 5/10
169/169 [==============================] - 54s 321ms/step - loss: 0.1786 - accuracy: 0.9378
Epoch 6/10
169/169 [==============================] - 56s 333ms/step - loss: 0.1492 - accuracy: 0.9460
Epoch 7/10
169/169 [==============================] - 56s 329ms/step - loss: 0.1276 - accuracy: 0.9530
Epoch 8/10
169/169 [==============================] - 54s 317ms/step - loss: 0.0967 - accuracy: 0.9671
Epoch 9/10
169/169 [==============================] - 53s 314ms/step - loss: 0.0942 - accuracy: 0.9679
Epoch 10/10
169/169 [==============================] - 55s 322ms/step - l

In [25]:
model.save('fruitModelWatson.h5')

In [26]:
!tar -zcvf IBM_TrainedModel.tgz fruitModelWatson.h5

fruitModelWatson.h5


In [24]:
!ls

fruit-dataset  fruitModelWatson  IBM_TrainedModel.tgz


In [27]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 22.0 MB/s eta 0:00:01


In [29]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "TAMGDjl2zVuiL_u957FqwDQSRSGpEediiqRVP6o_8zHS"
}

client = APIClient(wml_credentials)

In [66]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [71]:
space_uid = guid_from_space_name(client, 'communi_1_deployment')
print("Space UID : ", space_uid)

In [74]:
client.set.default_space(space_uid)

Note: you may need to restart the kernel to use updated packages.


In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='IBM_TrainedModel.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_uid(model_details)